<a href="https://colab.research.google.com/github/MikhailBuimov/pyspark/blob/main/laser_cutting_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pyspark
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StructType, StringType, IntegerType

In [3]:
from pyspark.sql import SparkSession

# Инициализация SparkSession
spark = (
    SparkSession.builder
    .appName("MyApp")
    .master("local")  # Запуск Spark на локальной машине
    .getOrCreate()
)

df = pd.read_csv('/content/drive/MyDrive/datasets/laser_cutting.csv')
psdf = ps.read_csv('/content/drive/MyDrive/datasets/laser_cutting.csv')

sqldf = spark.createDataFrame(df)

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [4]:
print(type(sqldf))

<class 'pyspark.sql.dataframe.DataFrame'>


In [6]:
print(type(psdf))

<class 'pyspark.pandas.frame.DataFrame'>


Я так и не понял, есть ли разница в pyspark.sql и pyspark.pandas. В доке написано, что это просто удобный интерфейс, чтобы делать все также как в пандасе(sql). Но по факту нет - получаются разные объекты. А когда я пытался дальше сделать udf с pyspark.pandas - оно не работало (withColumn не делался, потому что объект типа не pyspark. Хотя выше же тип pyspark.pandas!). Я так и не понимаю что тут происходит под капотом с этим pyspark

Тот что sql имеет свои методы (кривые какие то, неудобные. Вот переименовать столбцы разом хз как - очень неудобно) и выводит так

In [7]:
sqldf = sqldf.withColumnRenamed('Наименование', 'Part_name').withColumnRenamed('Наименование', 'Part_name')

sqldf.show()

+--------------------+--------------------+--------+----------+----------+------+-----+----------+------------+
|           Part_name|            Материал|Операция|      hash|     Время|Ширина|Длина|Длина реза|Кол-во резов|
+--------------------+--------------------+--------+----------+----------+------+-----+----------+------------+
|             Круг 40| Лист х/к т. 3,0 мм.|      ЛР|42af1b9fc3|00:00:03.5|  40.0| 40.0|       126|           1|
|             Круг 40| Лист х/к т. 3,0 мм.|      ЛР|42af1b9fc3|00:00:03.5|  40.0| 40.0|       126|           1|
|             Круг 40| Лист х/к т. 3,0 мм.|      ЛР|42af1b9fc3|00:00:03.5|  40.0| 40.0|       126|           1|
|             Круг 40| Лист х/к т. 3,0 мм.|      ЛР|42af1b9fc3|00:00:03.5|  40.0| 40.0|       126|           1|
|             Круг 40| Лист х/к т. 3,0 мм.|      ЛР|42af1b9fc3|00:00:03.5|  40.0| 40.0|       126|           1|
|             Круг 40| Лист х/к т. 3,0 мм.|      ЛР|42af1b9fc3|00:00:03.5|  40.0| 40.0|       126|      

А методы пандаса стандартные не имеет

In [8]:
sqldf.info()

AttributeError: 'DataFrame' object has no attribute 'info'

Буду делать через pyspark.pandas

Но причем тут тогда этот SparkSession?

Я не понимаю че происходит(



---



In [9]:
cutting = psdf

In [10]:
cutting.rename(columns={'Наименование': 'Part_name',
                        'Материал': 'Material',
                        'Время': 'Time',
                        'Ширина': 'Width',
                        'Длина': 'Length',
                        'Длина реза': 'Cut_length',
                        'Кол-во резов': 'Cut_quantity',
                        'Операция': 'Operation'}, inplace=True)

In [11]:
cutting.drop_duplicates(subset='hash', inplace=True)
len(cutting)

17830

In [12]:
import re

def get_keywords(name):
    # Определяем список предлогов и несущественных слов, которые нужно исключить
    stop_words = {'и', 'в', 'на', 'с', 'под', 'за',
                  'для', 'по', 'от', 'до', 'над',
                  'через', 'у', 'о', 'об', 'при',
                  'из', 'а', 'но', 'или', 'xx', 'хх',
                 'мм', 'ст', 'шт', 'тр', 'дл',
                 "шир", "выс", "пр"}

    name = re.sub(r'[\W_]', ' ', name)

    # Удаляем нетекстовые символы и числа
    name = re.sub(r'[^А-Яа-яЁё\s]', '', name)

    # Разбиваем строку на слова и удаляем лишние пробелы
    words = name.strip().split()

    # Фильтруем слова, исключая предлоги и слова, которые короче 2 символов
    keywords = [word.lower() for word in words if word.lower() not in stop_words and len(word) > 1]

    return keywords

In [13]:
cutting['keywords'] = cutting['Part_name'].apply(get_keywords)

In [14]:
print(type(cutting))

<class 'pyspark.pandas.frame.DataFrame'>


Ну вот получается pyspark объект. Но я просто копирую тот же код, что и в обычном питоновском EDA. Это точно то что надо??? Просто скопировать тот же код, зачем?

Ерунда какая-то...